In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import matplotlib as mpl
from cmcrameri import cm as cmc
import pandas as pd
import xarray as xr
import itertools


import pdb
import sys
sys.path.insert(0, '../lib')
sys.path.insert(0, '../src')
from scipy.stats import norm, beta, bernoulli


import ml_estimation as ml
import ml_estimation2 as ml2
import model1_plots as mp
import model1_explore as me
import model1_sim as sim1
import Utilities as util

In [5]:
## simulation results
method = 'main_beta'
T = 50
N = 40
X = xr.open_dataset(f'..\mod\model2\sim\simulation2_{method}_T{T}_N{N}_image_20201210-15h1457.nc')

In [4]:
## original images
loc_clean_data = '../data/clean_data/'
import pickle
from datetime import datetime 
with open(loc_clean_data + 'clean_dates.txt', 'rb') as f:
    dates = pickle.load(f)

# print(dates)
date = dates.date

start_date = datetime.strptime('10-12-2020', '%d-%m-%Y')
end_date = datetime.strptime('11-12-2020', '%d-%m-%Y')
idx = (start_date < date)  & (date < end_date) 

dates = dates[idx]

file = dates.file_name.loc[0]
images = xr.open_dataset(file)
images

# Only keep points which are in the right area and do contain a wind speed
dss = []
for file in dates.file_name:
    images = xr.open_dataset(file)
    dss.append(images)

images = xr.concat(dss, 't')

invalid =np.sum(images.ct == 0, axis = (1,2))
images = images.where(invalid < 10000, drop = True)

images = images.rename(dict(x = 'i', y = 'j'))
images = images.rename(dict(cth = 'h', cod = 'd'))

images['i'] = range(len(images.i))
images['j'] = range(len(images.j))
images['d'] = np.log(images.d)
images['z'] = (['t', 'i', 'j'], np.select([images.ct == 1, (images.ct > 1) & (~np.isnan(images.d)) & (~np.isnan(images.h))], [1, 0], np.nan))
# images['z_cloud'] = images.z.where(~np.isnan(images.d) & ~np.isnan(images.h))

images['d'] = images.d.where(images.z == 0)
images['h'] = images.h.where(images.z == 0)